In [1]:
import importlib
import pandas as pd
import numpy as np
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
""" 
Для формування списку ЄДРПОУ навчальних закладів, які потрібно наносити на мапу, використані бази даних МОН, опубліковані у листопаді 2019 року на Порталі відкритих даних.

### Перелік закладів загальної середньої та дошкільної освіти (ЗЗСДО)
    https://data.gov.ua/dataset/bd70757a-ab49-46be-a650-608a9b3bd7b1
    
### Перелік закладів позашкільної освіти (державна форма власності)
    https://data.gov.ua/dataset/9a88fb96-a043-4769-acf1-1c4a948ec285
    Перелік закладів позашкільної освіти системи Міністерства освіти і науки України
    https://data.gov.ua/dataset/18d7fb70-3781-4017-88d2-651e14d14211
    Заклади професійної (професійно-технічної) освіти
    https://data.gov.ua/dataset/824513a9-aa5e-45c8-b45e-dced6909761a
"""

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [3]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns.tolist()
print("Дані з файлу викачки BIPro Prozorro: " + "df_raw" + str(df_raw.shape))

Дані з файлу викачки BIPro Prozorro: df_raw(537893, 11)


##### Step 0: Підготовка бази МОН для використання у фільтруванні закупівель

In [4]:
expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';', low_memory = False)
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';', low_memory = False)

In [5]:
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])
print("expdata_sc", expdata_sc.shape)

expdata_sc (15206, 21)


In [6]:
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])
print("expdata_dnz", expdata_dnz.shape)

expdata_dnz (15745, 21)


In [7]:
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)
print("База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data", df_mon_data.shape)

База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data (30951, 22)


In [8]:
filter_keywords_exclude = ["департамент", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_raw[df_raw['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_exclude)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()
print("edrpou_upravlin_osvity_prozorro", len(edrpou_upravlin_osvity_prozorro))

mask = (df_mon_data['edrpou'].isin(edrpou_upravlin_osvity_prozorro))
df_mon_data_bez_upravlin_osvity = df_mon_data.loc[~mask]
print("df_mon_data_bez_upravlin_osvity", df_mon_data_bez_upravlin_osvity.shape)

/home/artem/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


edrpou_upravlin_osvity_prozorro 234
df_mon_data_bez_upravlin_osvity (30881, 22)


In [ ]:
"""
TODO:
    Артем: 
        створити базу управлінь освіти для формування переліку закупівель управлінь освіти 
        (закупівлі можуть дублюватись із закупівлями шкіл і садків)
        ЄДРПОУ - Місто (для Києва - по районам, якщо можливо)

"""

##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro. Виділення закупівель шкіл-замовників , згідно з базою МОН (пошук по колонці "IDOrganizator"

In [40]:
mon_data = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()
# mon_data = get_mon_data()

df_tenders_by_schools_by_edrpou = df_raw[df_raw['IDOrganizator'].isin(mon_data)]
df_undefined_1 = df_raw[~df_raw['IDOrganizator'].isin(mon_data)]

print("Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou " + str(df_tenders_by_schools_by_edrpou.shape))
print("Невизначені записи: df_undefined_1 " + str(df_undefined_1.shape))

Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou (32188, 11)
Невизначені записи: df_undefined_1 (505705, 11)


In [ ]:
# evaluation of results of Step 1

# df_tenders_by_schools_by_edrpou['Организатор'].value_counts()

In [14]:
# Experiments Step 1

school_names = df_mon_data_bez_upravlin_osvity.university_name.tolist()
df_tenders_by_schools_by_name = df_raw[df_raw['Организатор'].isin(school_names)]
df_tenders_by_schools_by_name.shape

(5888, 11)

In [55]:
print(df_mon_data_bez_upravlin_osvity.columns)

Index(['university_name', 'edrpou', 'university_type_name',
       'education_type_name', 'university_level', 'location_type',
       'university_financing_type_name', 'post_index', 'koatuu_id', 'language',
       'koatuu_full_name', 'university_address', 'post_index_u', 'koatuu_id_u',
       'region_name_u', 'koatuu_name_u', 'university_address_u',
       'university_phone', 'university_email', 'university_site',
       'university_director_fio', 'koatuu_name'],
      dtype='object')


In [56]:
cols = ['edrpou', 'university_name', 'university_type_name', 'education_type_name']
df_mon_data_bez_upravlin_osvity_for_merge = df_mon_data_bez_upravlin_osvity[cols]

df_tenders_by_schools_by_edrpou_extended = pd.merge(df_tenders_by_schools_by_edrpou, df_mon_data_bez_upravlin_osvity_merge, 
                                                                                 left_on='IDOrganizator',
                                                                                 right_on='edrpou',
                                                                                 how='left') 
df_tenders_by_schools_by_edrpou_extended.sample(3)

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,edrpou,university_name,university_type_name,education_type_name
14513,UA-2018-06-01-002621-a-L1,UA-2018-06-01-002621-a,"КОМУНАЛЬНИЙ ЗАКЛАД ""ЗАГАЛЬНООСВІТНЯ ШКОЛА І-ІІ...",55240000-4 Послуги центрів і будинків відпочин...,50960,NaN,55240000-4 Послуги центрів і будинків відпочин...,"Максима Шимка,3","ВУЛИЦЯ МАКСИМА ШИМКА, будинок 3",Вінницька міська рада,26119498,26119498,"Комунальний заклад ""Загальноосвітня школа І-ІІ...",заклад середньої освіти,загальносвітня школа
24118,UA-2019-01-03-001032-c-L1,UA-2019-01-03-001032-c,"КЗО ""НВК"" Вальдорфська середня загальноосвітня...",Теплова енергія,390253,Теплова енергія,Теплова енергія,проспект Олександра Поля б.133,"м. Дніпропетровськ, пр.Кірова, б.133",Дніпровська міська рада,23928928,23928928,Комунальний заклад освіти «Навчально-виховний ...,заклад дошкільної освіти,(null)
15676,UA-2018-12-22-003737-b-L2,UA-2018-12-22-003737-b,Житомирський навчально-виховний комплекс № 25,Сир кисломолочний та сир сичужний (твердий),21402.48,NaN,Сир сичужний (твердий),вул. Бориса Тена 84-а,вул.Бориса Тена 84-а,Житомирська міська рада,6669661,6669661,Житомирський навчально-виховний комплекс № 25,заклад середньої освіти,навчально-виховний комплекс (об'єднання)


##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [ ]:
df_tenders_by_schools_by_keywords = df_undefined_1[df_undefined_1['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), case = False, na = False, regex = True) ]
print(f"Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords {df_tenders_by_schools_by_keywords.shape}")

In [ ]:
# evalutaion #1 of results of Step 2
# df_tenders_by_schools_by_keywords['Организатор'].value_counts()

In [ ]:
# solution for Step 2

filter_keywords_step_2_exclude = ['професійн', 'міжшкільн']
df_tenders_by_schools_by_keywords_2_temp = df_tenders_by_schools_by_keywords[~df_tenders_by_schools_by_keywords['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_exclude)), case = False, na = False, regex = True)]
filter_keywords_step_2_include = ["школа І-І.*ступенів", "навчальн.*виховн.*комплекс", "гімназ", "ліце", "дошкільний"]
df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords_2_temp[df_tenders_by_schools_by_keywords_2_temp['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_include)), case = False, na = False, regex = True)]
print(f"Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів, {df_tenders_by_schools_by_keywords_2_temp.shape} \nДодаткове фільтрування датафрейму: {filter_keywords_step_2_include}. Залишилось:, {df_tenders_by_schools_by_keywords_2.shape}")

In [ ]:
# evalutaion #2 of results of Step 2

# df_tenders_by_schools_by_keywords_2_temp['Организатор'].value_counts()
# df_tenders_by_schools_by_keywords_2['Организатор'].value_counts()

##### Step 2.5: Об'єднання двох датафреймів шкіл-замовників в один

In [ ]:
df_tenders_by_schools = pandas.concat([df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_2])

print(f"Закупівлі шкіл-замовників: df_tenders_by_schools {df_tenders_by_schools.shape} \nНевизначені записи: df_undefined_1:{df_undefined_1.shape}")

##### Step 3: Класифікація шкіл-замовників по типах

In [ ]:
columns_to_classify = ['Организатор']
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, columns_to_classify, filter_priority, filter_name_priority)

In [ ]:
# evaluation of Step 3

# pd.set_option("display.max_colwidth", 200)
# print("Заклади, де не визначений тип:")
# df_tenders_by_schools_classified[df_tenders_by_schools_classified['Тип закладу'] == 'Невідомо']['Организатор'].unique().tolist()

##### Step 4: Нумерація шкіл-замовників

In [ ]:
columns_to_numerate = ['Организатор']
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, columns_to_numerate, reg_number)

##### Step 4.5: Найменування шкіл-замовників

In [ ]:
columns_to_name = ['Организатор']
df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, columns_to_name, reg_quotes)

In [ ]:
print('filter_poza', )
print('Було закупівель', df_tenders_by_schools_named.shape)
df_tenders_by_schools_named = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)),
                case = False, na = False, regex = True)]
df_tenders_by_schools_named['Номер'] = df_tenders_by_schools_named['Номер'].replace(r'^\s*$', np.nan, regex=True) # перетворити пусті строки в NaN
print(f'Залишилось закупівель після видалення закупівель закладів ключовими словами {filter_poza}, \ndf_tenders_by_schools_named', df_tenders_by_schools_named.shape)

In [ ]:
# # evaluation of Step 4

# for_analysis_step_4 = df_tenders_by_schools_named[df_tenders_by_schools_named['Назва'].isnull() & 
#                                   df_tenders_by_schools_named['Номер'].isnull()]
# print('Навчальні заклади, де не визначились номер чи назва:')
# for_analysis_step_4['Организатор'].unique().tolist()

##### Step 4.9: Форматування бази шкіл-замовників

In [ ]:
df_tenders_by_schools = df_tenders_by_schools_named.copy()
df_tenders_by_schools['Заклад-замовник'] = True

col_names = training_data_columns.copy() 
col_names += ['Номер', 'Назва', 'Тип закладу', 'Заклад-замовник']

df_tenders_by_schools = df_tenders_by_schools[col_names]

##### Step 5: Виділення закупівель шкіл-не-замовників за допомогою пошуку ключових слів в колонках Тендер, ОписаниеТендера, Лот

In [ ]:
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True)]

print("Закупівлі шкіл-не-замовників, визначених через Тендер, Лот, ОписаниеТендера: df_tenders_by_schools_by_keywords " + str(df_tenders_not_by_schools_by_keywords.shape))

In [ ]:
# evaluation of Step 5

# df_tenders_not_by_schools_by_keywords

##### Step 6: Класифікація шкіл-не-замовників по типах

In [ ]:
# тимчасово винсені з filters_lists фільтри сюди для тестування порядковості

filter_priority = [
        filter_nvk,
        filter_gimnasium,
        filter_liceum,
        filter_specializovana,
        filter_internat,
        filter_kombinat,
        filter_sanatorna,
        filter_specializovana,
        filter_specialna,
        filter_sadik,
        filter_shkola 
    ]

filter_name_priority = [
        filter_nvk_name,
        filter_gimnasium_name,
        filter_liceum_name,
        filter_specializovana_name,
        filter_internat_name,
        filter_kombinat_name,
        filter_sanatorna_name,
        filter_specializovana_name,
        filter_specialna_name,
        filter_sadik_name,
        filter_shkola_name
    ]

In [ ]:
columns_to_classify = ['Тендер', 'ОписаниеТендера', 'Лот']
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords, columns_to_classify, filter_priority, filter_name_priority)

In [ ]:
# # evaluation of Step 6

# pd.set_option("display.max_colwidth", 200)

# type_sample = 'Інтернат'
# df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] == type_sample][['Тендер', 'ОписаниеТендера', 'Лот', 'Тип закладу']].sort_values(by=['Тип закладу'])
# df_tenders_not_by_schools_classified['Тип закладу'].value_counts()

##### Step 7: Нумерація шкіл-не-замовників

In [ ]:
"""
Принцип фільтрування в функції numeration():

    Визначається наявністья символу № в кожній колонці columns_to_numerate_step7
    
    Регулярний вираз знаходить всі паттерни і джойнить через кому
    
    Паралельно створюється колонка, де рахується кількість знайдених паттернів
    
    Результат фільтрування numeration() - потрапляє у датафрейм df_step7_numerated
"""
columns_to_numerate_step7 = ['Тендер', 'Лот']
action_mark = 'numeration'
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

df_step7_numerated = numeration(df_step7, columns_to_numerate_step7, reg_number, action_mark)
print("df_step7_numerated", df_step7_numerated.shape)

In [ ]:
"""
    Якщо в колонках Лот і Тендер знайшло по одному паттерну і вони однакові - 
    закупівля вважається такою, де коректно визначений номер

    Ці закупівлі потрапляють до датафрейму df_step7_numerated_lot_tender

    Колонка "Номер" отримує знайдений номер "Лот_numeration"

    Решта закупівель потрапляють до датафрейму df_step7_numerated_undefined.
"""

df_step7_numerated['Номер'] = np.where(((df_step7_numerated['Тендер_numeration_count'] == 1) | 
                                        (df_step7_numerated['Лот_numeration_count'] == 1)) & 
                                        (df_step7_numerated['Лот_numeration_count'] == df_step7_numerated['Тендер_numeration_count']), 
                                        df_step7_numerated['Лот_numeration'], 
                                        'Невідомо')

df_step7_numerated_lot_tender = df_step7_numerated[df_step7_numerated['Номер'] != 'Невідомо']

df_step7_numerated_undefined = df_step7_numerated[df_step7_numerated['Номер'] == 'Невідомо']

print("df_step7_numerated_lot_tender", df_step7_numerated_lot_tender.shape, "\ndf_step7_numerated_undefined", df_step7_numerated_undefined.shape)

In [ ]:
"""
    Якщо в колонці Лот знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_lot
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_2
"""

df_step7_numerated_lot = df_step7_numerated_undefined.copy()

df_step7_numerated_lot['Номер'] = np.where(df_step7_numerated_lot['Лот_numeration_count'] == 1, 
                                                 df_step7_numerated_lot['Лот_numeration'], 
                                                 'Невідомо')

df_step7_numerated_undefined_2 = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] == 'Невідомо']
df_step7_numerated_lot = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] != 'Невідомо']

print("df_step7_numerated_lot", df_step7_numerated_lot.shape, "\ndf_step7_numerated_undefined_2", df_step7_numerated_undefined_2.shape)

In [ ]:
"""
    Якщо в колонці Тендер знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_tender
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_3
"""
df_step7_numerated_tender = df_step7_numerated_undefined_2.copy()

df_step7_numerated_tender['Номер'] = np.where(df_step7_numerated_tender['Тендер_numeration_count'] == 1, 
                                                 df_step7_numerated_tender['Тендер_numeration'], 
                                                 'Невідомо')
df_step7_numerated_undefined_3 = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] == 'Невідомо']
df_step7_numerated_tender = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] != 'Невідомо']

print("df_step7_numerated_tender", df_step7_numerated_tender.shape, "\ndf_step7_numerated_undefined_3", df_step7_numerated_undefined_3.shape)

##### Step 7.5: Об'єднання датафреймів шкіл-не-замовників з номерами

In [ ]:
frames = [df_step7_numerated_lot_tender, df_step7_numerated_lot, df_step7_numerated_tender]
df_tenders_not_by_schools_numerated = pd.concat(frames)
df_tenders_not_by_schools_numerated['Назва'] = np.nan # Створення пустої колонки "Назва" для подальшого обєднання з датафреймом з номерами

##### Step 8: Найменування шкіл-не-замовників

In [ ]:
columns_to_classify_step8 = ['Тендер', 'Лот']
action_mark = 'naming'
df_step8 = df_step7_numerated_undefined_3.copy()

df_tenders_not_by_schools_named = naming(df_step8, columns_to_classify_step8, reg_quotes, action_mark)
print("df_tenders_not_by_schools_named", df_tenders_not_by_schools_named.shape)

In [ ]:
df_tenders_not_by_schools_named['Назва'] = np.where((df_tenders_not_by_schools_named['Тендер_naming'].notnull()) & 
                             (df_tenders_not_by_schools_named['Лот_naming'].notnull()) & 
                             (df_tenders_not_by_schools_named['Тендер_naming'] == df_tenders_not_by_schools_named['Лот_naming']),
                             df_tenders_not_by_schools_named['Тендер_naming'], 
                             'Невідомо')

df_step8_undefined = df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Назва'] == 'Невідомо']
df_tenders_not_by_schools_named_lot_tender = df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Назва'] != 'Невідомо']

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [ ]:
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)

In [ ]:
df_tenders_not_by_schools['Заклад-замовник'] = False

In [ ]:
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]
print("df_tenders_not_by_schools_numerated", df_tenders_not_by_schools_numerated.shape)

In [ ]:
df_tenders_not_by_schools

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [ ]:
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)
print("df_tenders", df_tenders.shape)

#### Step 11: RandomDataResuls

In [67]:
"""
Колонка Correct:
    1 - коректо визначено
    
    Некоректно:
        2 - номер визначений неправильно:
            
            21 - відсутній символ №
            22 - кілька символів №
            23 - без лапок
            24 - номер чи назва в колонці Описание Тендер
            25 - лапки не визначились
            26 - кілька лапок
        
        3 - неправильно тип закладу
"""

random_1000_results = pandas.read_csv('results/school-procurements-1000-random-marked.csv', index_col = 0, error_bad_lines=False, sep = ',', low_memory = False)
random_1000_results = random_1000_results.iloc[:, :-1]
print(random_1000_results.shape)

(1000, 8)


In [68]:
random_1000_results

,Тендер,ОписаниеТендера,Лот,Организатор,Тип закладу,Номер/Назва,Заклад-замовник,Correct
445840,Капітальний ремонт і реставрація:Капітальний р...,ДСТУ Б Д.1.1-1:2013,Капітальний ремонт і реставрація:Капітальний р...,Департамент освіти Харківської міської ради,ДНЗ,№400,False,1
166960,Капітальний ремонт (відновлення) груп дошкільн...,Капітальний ремонт (відновлення) груп дошкільн...,Капітальний ремонт (відновлення) груп дошкільн...,Управління будівництва Дніпровської районної в...,ДНЗ,№ 680,False,1
180001,Перевірка 3-ф ел.ліч.прям.вмикання в СНВК № 86,NaN,Перевірка 3-ф ел.ліч.прям.вмикання в СНВК № 86,ВК Управління освіти виконавчого комітету Полт...,Навчально-виховний комплекс (об'єднання),№ 86,False,1
61147,Розроблення проектно-кошторисної документації ...,NaN,Розроблення проектно-кошторисної документації ...,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,ДНЗ,№302,False,1
158417,Поточний ремонт внутрішньої мережі центральног...,NaN,Поточний ремонт внутрішньої мережі центральног...,Управління освіти Дніпровської районної в міст...,Інтернат,№14,False,1
...,...,...,...,...,...,...,...,...
65456,Гардини тюлеві готові,"Посадові особи замовника, уповноважені здійсню...",Гардини тюлеві готові,"Загальноосвітній навчальний заклад ""Київський ...",Загальноосвітня школа,Київський спортивний ліцей-інтернат,True,3
278714,"Хлібопродукти, свіжовипечені хлібобулочні та к...",NaN,"Хлібопродукти, свіжовипечені хлібобулочні та к...","КОМУНАЛЬНИЙ ЗАКЛАД ""ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛ...",ДНЗ,№ 13,True,1
92556,Послуги з ремонту і технічного обслуговування ...,поточний ремонт та обслуговування системи опал...,Послуги з ремонту і технічного обслуговування ...,"КУ Сумський навчально-виховний комплекс ""загал...",Загальноосвітня школа,№ 11,True,3
237438,Рукавички гумові.,Рукавиці універсальні надміцніФрекен БОК або а...,Рукавички гумові.,"Загальноосвітній навчальний заклад ""Київський ...",Загальноосвітня школа,Київський спортивний ліцей-інтернат,True,3
